In [1]:
import os
import uuid
import json
import logging
from dotenv import load_dotenv

from dapr_agents import OpenAIChatClient
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from dapr_agents.tool import AgentTool

logging.basicConfig(level=logging.INFO)
load_dotenv()

True

In [2]:
llm = OpenAIChatClient(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

def to_obj(x):
    return json.loads(x) if isinstance(x, str) else x

INFO:dapr_agents.llm.openai.client.base:Initializing Azure OpenAI client...
INFO:dapr_agents.llm.openai.client.azure:Using API key for authentication.


In [ ]:
# ---- MCP Jupyter server env (REMOTE) ----
NOTEBOOK_NAME = f"mcp_{uuid.uuid4().hex[:8]}.ipynb"

ENV = {
    **os.environ,
    "MCP_JUPYTER_SESSION_MODE": "server",
    "MCP_JUPYTER_BASE_URL": os.getenv("JUPYTER_BASE_URL"),
    "MCP_JUPYTER_TOKEN": os.getenv("JUPYTER_TOKEN"),
    "MCP_JUPYTER_KERNEL_NAME": os.getenv("JUPYTER_KERNEL_NAME", "python3"),
    "MCP_JUPYTER_NOTEBOOK_PATH": os.getenv("JUPYTER_NOTEBOOK_PATH", NOTEBOOK_NAME),
}

# Adjust path to your MCP server entrypoint (where your server.py lives)
server_params = StdioServerParameters(
    command="uvx",
    args=[ "mcp-jupyter-notebook", "--transport", "stdio"],
    env=ENV,
)

In [4]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as s:
        await s.initialize()

        # Discover tools
        tools = await AgentTool.from_mcp_session(s)

        from dapr_agents import Agent

        notebook_agent = Agent(
            name="Jupyter Notebook Orchestrator",
            role="A notebook co-pilot that drives analysis inside a persistent Jupyter session.",
            goal=(
                "Use the active Jupyter notebook to complete tasks end-to-end: "
                "add clear markdown, execute Python code, and, when necessary, install Python packages. "
                "Reuse the same live session (kernel + document) so work builds on previous cells."
            ),
            instructions=[
                # Session & ordering
                "You are driving a live Jupyter notebook (persistent kernel + document).",
                "Cell order matters. Issue exactly ONE tool call at a time and wait for its result before the next.",
                "Append new cells; do not rewrite or reorder existing cells unless explicitly asked.",

                # Tools to use
                "Use `notebook.markdown.add(content)` for headings and short explanations.",
                "Use `notebook.code.run(content)` to append and execute Python code cells.",

                # Packages
                "If an import fails, call `notebook.packages.add([pip_names])` once, then retry the import.",
                "Package names are pip distributions (e.g., 'matplotlib', 'seaborn', 'plotly', 'requests').",

                # Notebook hygiene
                "Before a task, add a brief markdown heading describing what you'll do.",
                "Keep code cells small and focused—one idea per cell.",
                "When previewing data, show concise outputs (e.g., df.head(), shapes, key metrics).",
                "For plots, include a title, axis labels, and legends where helpful.",

                # Robustness
                "If an error occurs, add a short markdown note on the fix, then run a corrected code cell.",
                "Reuse variables and data created in earlier cells; reference them directly in later cells.",

                # Closure
                "End a logical section with a short markdown summary of what was produced."
            ],
            llm=llm,
            tools=tools,
            max_iterations=20,
        )

        QUESTION = """create a matplotlib bar and line chart examples in the notebook"""
        await notebook_agent.run(QUESTION)

INFO:dapr_agents.tool.executor:Tool registered: Notebook.Markdown.Add
INFO:dapr_agents.tool.executor:Tool registered: Notebook.Code.Run
INFO:dapr_agents.tool.executor:Tool registered: Notebook.Packages.Add
INFO:dapr_agents.tool.executor:Tool Executor initialized with 3 tool(s).


user:
create a matplotlib bar and line chart examples in the notebook

--------------------------------------------------------------------------------


INFO:dapr_agents.agents.agent.agent:Iteration 1/20 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Calling OpenAI ChatCompletion...


INFO:httpx:HTTP Request: POST https://bh25-shared-azfoundry-w3.cognitiveservices.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:ChatCompletion response received.


assistant:
Function name: Notebook.Markdown.Add (Call Id: call_a8ON6swQR5M90M8Kr0WxnTrN)
Arguments: {"content":"# Bar Chart and Line Chart Examples using Matplotlib\n\nIn this section, I will create examples of a bar chart and a line chart using the matplotlib library."}

--------------------------------------------------------------------------------


...


Notebook.Code.Run(tool) (Id: call_MHgn1jsTsZhdB89kWj3drGkD):
{"ok":true,"cell_index":2,"status":"ok","stdout":"","stderr":"","outputs":[{"output_type":"display_data","data":{"text/plain":"<Figure size 800x400 with 1 Axes>","image/png":"iVBORw0KGgoAAAANSUhEUgAAAq4AAAGJCAYAAABLvrEVAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjYsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvq6yFwwAAAAlwSFlzAAAPYQAAD2EBqD+naQAANGBJREFUeJzt3XlYVHXj///XgDAgm4oKoriUC2pZZqm45IaRuWSRZllp2o6mcmdJi5pppLllIVYfhMzMsrS0+1Zvw9IWl6JMKzUtFwoGswREbwaC8/ujr/NzQhMMOBx9Pq7rXFfnfZZ5nXFu7pdvzxxshmEYAgAAAKo5D7MDAAAAAGVBcQUAAIAlUFwBAABgCRRXAAAAWALFFQAAAJZAcQUAAIAlUFwBAABgCRRXAAAAWALFFQAAAJZAcQWA85CamiqbzaYvv/zS7CgXraZNm2rkyJFmxwBQhSiuAEx3qgSevtSvX1+9evXS2rVrqzzPqlWr1K9fP9WtW1fe3t4KCwvT0KFDtXHjxirPcsrChQuVmppa5v3/+n6evjzwwAOVFxQAKlENswMAwCnTpk1Ts2bNZBiGsrOzlZqaqhtuuEFr1qzRgAEDKv31DcPQqFGjlJqaqvbt2ysuLk6hoaHKysrSqlWr1KdPH3322Wfq0qVLpWf5q4ULF6pu3brlmmHs27ev7rrrrlLjLVu2rMBkAFB1KK4Aqo1+/frp6quvdq2PHj1aISEhevPNNyukuJaUlKiwsFA+Pj5n3D5nzhy

INFO:dapr_agents.agents.agent.agent:Iteration 3/20 started.
INFO:dapr_agents.llm.utils.request:Tools are available in the request.
INFO:dapr_agents.llm.openai.chat:Calling OpenAI ChatCompletion...


INFO:httpx:HTTP Request: POST https://bh25-shared-azfoundry-w3.cognitiveservices.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:ChatCompletion response received.


assistant:
I have created examples of a bar chart and a line chart using matplotlib. The bar chart displays categorical values, and the line chart shows the same data points connected with lines and markers. Both charts are labeled with titles and axis labels for clarity. Would you like me to add any other types of charts or modify these examples?

--------------------------------------------------------------------------------


INFO:dapr_agents.agents.agent.agent:Agent conversation completed after 3 turns.
